<h4> Enviroment Setup</h4>

In [2]:
# modules we'll use
import pandas as pd
import numpy as np

# helpful modules
import fuzzywuzzy
from fuzzywuzzy import process
import charset_normalizer

# read in all our data
professors = pd.read_csv("../data/pakistan_intellectual_capital.csv")

# set seed for reproducibility
np.random.seed(0)


c:\Users\yehia\AppData\Local\Programs\Python\Python313\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [3]:
countries=professors['Country'].unique()

countries.sort()

countries


array([' Germany', ' New Zealand', ' Sweden', ' USA', 'Australia',
       'Austria', 'Canada', 'China', 'Finland', 'France', 'Greece',
       'HongKong', 'Ireland', 'Italy', 'Japan', 'Macau', 'Malaysia',
       'Mauritius', 'Netherland', 'New Zealand', 'Norway', 'Pakistan',
       'Portugal', 'Russian Federation', 'Saudi Arabia', 'Scotland',
       'Singapore', 'South Korea', 'SouthKorea', 'Spain', 'Sweden',
       'Thailand', 'Turkey', 'UK', 'USA', 'USofA', 'Urbana', 'germany'],
      dtype=object)

<p>As we see here in Country column their are inconsistent data we can see for an example ( New Zealand) and (New Zealand) also (Germany) and (germany)</p>

We can Do some modification and cleaning in the data we observed

In [4]:
# convert to lower case
professors['Country']=professors["Country"].str.lower()
#reomve trailing white spaces
professors['Country']=professors['Country'].str.strip()

<p>Fuzzy matching: The process of automatically finding text strings that are very similar to the target string. In general, a string is considered "closer" to another one the fewer characters you'd need to change if you were transforming one string into another. So "apple" and "snapple" are two changes away from each other (add "s" and "n") while "in" and "on" and one change away (rplace "i" with "o"). You won't always be able to rely on fuzzy matching 100%, but it will usually end up saving you at least a little time</p>

In [5]:
# get the top 10 closest matches to "south korea"
matches = fuzzywuzzy.process.extract("south korea", countries, limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)

# take a look at them
matches

[('South Korea', 100),
 ('SouthKorea', 48),
 ('Saudi Arabia', 43),
 ('Norway', 35),
 ('Ireland', 33),
 ('Portugal', 32),
 ('Singapore', 30),
 ('Netherland', 29),
 ('Macau', 25),
 ('USofA', 25)]

In [6]:
def replace_matches_in_column(df,column,string_to_match,min_ratio=47):
    # get a list of unique strings
    strings=df[column].unique()
    
    # get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(string_to_match, strings, 
                                         limit=10, scorer=fuzzywuzzy.fuzz.token_sort_ratio)
    
    # only get matches with a ratio > 90
    close_matches=[matches[0]for matches in matches if matches[1]>=min_ratio]
    # get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # replace all rows with close matches with the input matches 
    df.loc[rows_with_matches, column] = string_to_match    

In [7]:
# use the function we just wrote to replace close matches to "south korea" with "south korea"
replace_matches_in_column(df=professors, column='Country', string_to_match="south korea")